# Grappling with a CNN

This notebook will convince me that I can build a CNN.

In [93]:
# Libraries
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import tensorflow.keras as keras
import tensorboard
import psycopg2
from PIL import Image
import os
import h5py

The data are stored locally in a PostgreSQL database. So let's connect and get them.

In [94]:
# Read in data from origami database in PostgreSQL
try:
    print('Attempting to connect to PostgreSQL.')
    conn = None
    conn = psycopg2.connect(host='localhost', database='origami', user='postgres', password='postgres')
    cur = conn.cursor()
    # SELECT image classifications and file paths from PostgreSQL
    sql_select = "SELECT image_class, image_path FROM origami_images;"
    cur.execute(sql_select)
    print('Selecting rows from origami_images table.')
    origami_images = cur.fetchall()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        cur.close()
        conn.close()
        print('PostgreSQL connection closed.')

Attempting to connect to PostgreSQL.
Selecting rows from origami_images table.
PostgreSQL connection closed.


This gives us `origami_images`, a list of tuples, each of which gives a class and a file path for the associated image.

In [95]:
print(len(origami_images))
print(origami_images[0])

166
('butterfly', '/Users/wwatson/Desktop/Insight/Project/origami/Images/downloads/butterfly/26.jpeg')


Let's shuffle this list randomly.

In [96]:
random.shuffle(origami_images)

We'll turn this into two `numpy` arrays to train our model. The class names are easy enough.

In [97]:
origami_y = []
class_names = []
for img in origami_images:
    origami_y.append(img[0])
    if img[0] not in class_names:
        class_names.append(img[0])
origami_y = np.array(origami_y)
class_names = np.array(class_names)
print(class_names)
print(origami_y)

['star' 'duck' 'crane' 'frog' 'butterfly']
['star' 'duck' 'star' 'crane' 'star' 'crane' 'frog' 'star' 'crane' 'duck'
 'butterfly' 'crane' 'star' 'crane' 'butterfly' 'duck' 'duck' 'frog'
 'crane' 'frog' 'crane' 'frog' 'crane' 'frog' 'frog' 'star' 'frog' 'duck'
 'duck' 'butterfly' 'duck' 'star' 'butterfly' 'crane' 'duck' 'butterfly'
 'star' 'crane' 'duck' 'frog' 'frog' 'frog' 'star' 'frog' 'butterfly'
 'duck' 'butterfly' 'star' 'star' 'crane' 'star' 'duck' 'frog' 'star'
 'frog' 'frog' 'butterfly' 'butterfly' 'crane' 'butterfly' 'butterfly'
 'star' 'crane' 'butterfly' 'frog' 'duck' 'duck' 'butterfly' 'frog'
 'butterfly' 'butterfly' 'crane' 'butterfly' 'butterfly' 'frog' 'star'
 'butterfly' 'crane' 'duck' 'duck' 'duck' 'star' 'duck' 'butterfly' 'star'
 'star' 'duck' 'crane' 'duck' 'star' 'butterfly' 'frog' 'duck' 'butterfly'
 'duck' 'crane' 'butterfly' 'star' 'duck' 'butterfly' 'frog' 'duck'
 'crane' 'crane' 'duck' 'frog' 'butterfly' 'crane' 'frog' 'star' 'crane'
 'star' 'duck' 'duck' 'cra

The images are a little harder. Let's define a couple of functions to help with image processing.

In [98]:
# Side length of normalized image
IMG_SIZE = 256

# Function to decode an image, render in grayscale and square dimensions
def decode_img(img):
    img = tf.io.decode_image(img, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float64)
    img = tf.image.rgb_to_grayscale(img)
    return tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

# Function to process a file path and return the image
def process_path(img_path):
    # Decode image using auxiliary function
    img = tf.io.read_file(img_path)
    img = decode_img(img)
    return img

OK, so now let's create our second array.

In [ ]:
origami_X = []
counter = 0
for img in origami_images:
    #print(img[1])
    origami_X.append(process_path(img[1]))
    counter += 1
    if counter % 10 == 0:
        print('{} files processed.'.format(counter))
print('All files processed. Converting to tensor.')
origami_X = tf.convert_to_tensor(origami_X)

10 files processed.
20 files processed.
30 files processed.
40 files processed.
50 files processed.
60 files processed.
70 files processed.
80 files processed.
90 files processed.
100 files processed.
110 files processed.
120 files processed.


OK, let's give this CNN thing a whirl...

In [ ]:
cnn = keras.Sequential()
# Layer construction w/ L2 regularizers on convolutional layers
cnn.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
cnn.add(keras.layers.MaxPooling2D((2, 2)))
cnn.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(keras.layers.Flatten())
cnn.add(keras.layers.Dense(16, activation='relu'))
cnn.add(keras.layers.Dense(len(class_names), activation='softmax'))

Compile it.

In [ ]:
cnn.compile(optimizer='adam',
           loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
           metrics=[keras.losses.SparseCategoricalCrossentropy(from_logits=True, name='sparse_crossentropy'),
                    'categorical_accuracy'])
cnn.summary()

And fit it.

In [92]:
cnn.fit(origami_X, origami_y, epochs=10)

Train on 166 samples
Epoch 1/10
 32/166 [====>.........................] - ETA: 8s

UnimplementedError:  Cast string to int64 is not supported
	 [[node loss/dense_13_loss/Cast (defined at /Users/wwatson/opt/anaconda3/envs/origami/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_7282820]

Function call stack:
distributed_function


OK...maybe a boring old perceptron for now?

In [80]:
nn = keras.Sequential()
# Layer construction w/ L2 regularizers on convolutional layers
nn.add(keras.layers.Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 1)))
nn.add(keras.layers.Dense(128, activation='relu'))
nn.add(keras.layers.Dense(32, activation='relu'))
nn.add(keras.layers.Dense(len(class_names), activation='softmax'))

In [81]:
nn.compile(optimizer='adam',
           loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
           metrics=[keras.losses.SparseCategoricalCrossentropy(from_logits=True, name='sparse_crossentropy'),
                    'categorical_accuracy'])
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               8388736   
_________________________________________________________________
dense_10 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 165       
Total params: 8,393,029
Trainable params: 8,393,029
Non-trainable params: 0
_________________________________________________________________


In [82]:
nn.fit(origami_X, origami_y, epochs=10)

Train on 166 samples
Epoch 1/10
 32/166 [====>.........................] - ETA: 7s

UnimplementedError:  Cast string to int64 is not supported
	 [[node loss/dense_11_loss/Cast (defined at /Users/wwatson/opt/anaconda3/envs/origami/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_7275725]

Function call stack:
distributed_function
